In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def match_features(descriptors1, descriptors2, ratio_thresh=0.75):
    """
    Matches descriptors between two sets.

    Args:
        descriptors1 (np.array): Descriptors from the first image.
        descriptors2 (np.array): Descriptors from the second image.
        ratio_thresh (float): Lowe's ratio test threshold.

    Returns:
        list: Filtered DMatch objects.
    """
    if descriptors1 is None or descriptors2 is None:
        return []
    if descriptors1.dtype != np.float32: # BFMatcher (and FLANN LSH) might expect specific types
        descriptors1 = descriptors1.astype(np.float32)
    if descriptors2.dtype != np.float32:
        descriptors2 = descriptors2.astype(np.float32)

    # Determine descriptor type for matcher (ORB are binary, SIFT are float)
    is_binary_descriptor = False
    if descriptors1.dtype == np.uint8: # Common for ORB, BRIEF
        is_binary_descriptor = True

    # NORM_HAMMING for binary descriptors (ORB), NORM_L2 for float (SIFT)
    norm_type = cv2.NORM_HAMMING if is_binary_descriptor else cv2.NORM_L2
    bf = cv2.BFMatcher(norm_type, crossCheck=False) # crossCheck=True is stricter matching

    # Perform k-NN matching
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    # Apply Lowe's ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_thresh * n.distance:
            good_matches.append(m)
    return good_matches

In [ ]:
def reject_outliers_ransac(keypoints1, keypoints2, matches, reproj_thresh=3.0, confidence=0.99, max_iters=1000):
    """
    Rejects outlier matches using RANSAC with Fundamental Matrix estimation.

    Args:
        keypoints1 (list): Keypoints from the first image.
        keypoints2 (list): Keypoints from the second image.
        matches (list): List of DMatch objects (raw matches).
        reproj_thresh (float): Maximum allowed reprojection error to treat a point pair as an inlier.
        confidence (float): Desired confidence that the estimated matrix is correct.
        max_iters (int): Maximum number of iterations for RANSAC.

    Returns:
        tuple: (list of DMatch objects (inliers), np.array (mask of inliers), np.array (Fundamental Matrix))
    """
    if not matches:
        return [], [], None

    # Convert keypoints to cv2.Point2f (float) format
    pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Find Fundamental Matrix and inlier mask using RANSAC
    # F is the Fundamental Matrix
    # mask is a boolean array where 1 means inlier, 0 means outlier
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC, reproj_thresh, confidence, max_iters)

    # Filter matches based on the mask
    inlier_matches = []
    if mask is not None:
        for i, m in enumerate(matches):
            if mask[i] == 1:
                inlier_matches.append(m)

    return inlier_matches, mask.ravel(), F # .ravel() converts mask to 1D array

In [ ]:
def visualize_matches(img1, keypoints1, img2, keypoints2, matches, inlier_mask=None, title="Matches"):
    """
    Visualizes feature matches between two images.

    Args:
        img1 (np.array): First image (BGR).
        keypoints1 (list): Keypoints from the first image.
        img2 (np.array): Second image (BGR).
        keypoints2 (list): Keypoints from the second image.
        matches (list): List of DMatch objects.
        inlier_mask (np.array, optional): Boolean mask for inliers (1) and outliers (0).
        title (str): Title for the plot.
    """
    if inlier_mask is not None:
        # Draw inliers (green) and outliers (red)
        img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches, None,
                                      matchColor=(0, 255, 0), # Green for inliers
                                      singlePointColor=(255, 0, 0), # Red for outliers
                                      matchesMask=inlier_mask.tolist(), # Convert mask to list for cv2.drawMatches
                                      flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
    else:
        # Draw all matches in default color
        img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches, None,
                                      flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

    plt.figure(figsize=(18, 9))
    plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
def match_features_orb_bf(descriptors1, descriptors2, ratio_thresh=0.75):
    """
    Matches ORB descriptors between two sets using Brute-Force Matcher with NORM_HAMMING.
    Applies Lowe's ratio test.

    Args:
        descriptors1 (np.array): ORB Descriptors from the first image.
        descriptors2 (np.array): ORB Descriptors from the second image.
        ratio_thresh (float): Lowe's ratio test threshold.

    Returns:
        list: Filtered DMatch objects.
    """
    if descriptors1 is None or descriptors2 is None or len(descriptors1) == 0 or len(descriptors2) == 0:
        return []

    # ORB descriptors are binary, so NORM_HAMMING is appropriate
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

    # Perform k-NN matching (k=2 for ratio test)
    # Ensure descriptors are of type CV_32F or CV_8U, match them to be safe
    descriptors1_ = np.float32(descriptors1) if descriptors1.dtype != np.float32 else descriptors1
    descriptors2_ = np.float32(descriptors2) if descriptors2.dtype != np.float32 else descriptors2

    # Using try-except for knnMatch as it can sometimes fail if not enough descriptors
    try:
        matches = bf.knnMatch(descriptors1_, descriptors2_, k=2)
    except cv2.error as e:
        print(f"Error during knnMatch: {e}. Skipping match for this pair.")
        return []

    # Apply Lowe's ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_thresh * n.distance:
            good_matches.append(m)
    return good_matches

def reject_outliers_ransac(keypoints1, keypoints2, matches, reproj_thresh=3.0, confidence=0.99, max_iters=1000):
    """
    Rejects outlier matches using RANSAC with Fundamental Matrix estimation.

    Args:
        keypoints1 (list): Keypoints from the first image.
        keypoints2 (list): Keypoints from the second image.
        matches (list): List of DMatch objects (raw matches).
        reproj_thresh (float): Maximum allowed reprojection error to treat a point pair as an inlier.
        confidence (float): Desired confidence that the estimated matrix is correct.
        max_iters (int): Maximum number of iterations for RANSAC.

    Returns:
        tuple: (list of DMatch objects (inliers), np.array (mask of inliers), np.array (Fundamental Matrix))
    """
    if not matches or len(matches) < 8: # Need at least 8 points for Fundamental Matrix
        print("Not enough matches to perform RANSAC (need at least 8).")
        return [], [], None

    # Convert keypoints to cv2.Point2f (float) format
    pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Find Fundamental Matrix and inlier mask using RANSAC
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC, reproj_thresh, confidence, max_iters)

    # Filter matches based on the mask
    inlier_matches = []
    if mask is not None:
        for i, m in enumerate(matches):
            if mask[i] == 1:
                inlier_matches.append(m)

    return inlier_matches, mask.ravel(), F # .ravel() converts mask to 1D array


def visualize_matches(img1, keypoints1, img2, keypoints2, matches, inlier_mask=None, title="Matches"):
    """
    Visualizes feature matches between two images.

    Args:
        img1 (np.array): First image (BGR).
        keypoints1 (list): Keypoints from the first image.
        img2 (np.array): Second image (BGR).
        keypoints2 (list): Keypoints from the second image.
        matches (list): List of DMatch objects.
        inlier_mask (np.array, optional): Boolean mask for inliers (1) and outliers (0).
        title (str): Title for the plot.
    """
    if inlier_mask is not None:
        # Draw inliers (green) and outliers (red)
        img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches, None,
                                      matchColor=(0, 255, 0), # Green for inliers
                                      singlePointColor=(255, 0, 0), # Red for outliers
                                      matchesMask=inlier_mask.tolist(), # Convert mask to list for cv2.drawMatches
                                      flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
    else:
        # Draw all matches in default color
        img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches, None,
                                      flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

    plt.figure(figsize=(18, 9))
    plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# %% [markdown]
# ### Execution for Part 4

# %%
print("\n--- Running Feature Matching and Outlier Rejection (ORB & Brute-Force) ---")

# Iterate through the chosen display_pair_indices_for_matching
for i in display_pair_indices_for_matching:
    j = i + 1

    if j >= len(frames):
        continue # Should not happen with correct display_pair_indices_for_matching setup

    img1 = frames[i]
    img2 = frames[j]

    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    print(f"\nProcessing Frame Pair: {i} and {j}")

    # 1. Feature Detection & Description (using ORB)
    keypoints1, descriptors1 = detect_orb(gray1)
    keypoints2, descriptors2 = detect_orb(gray2)

    print(f"  - Frame {i}: {len(keypoints1)} ORB keypoints")
    print(f"  - Frame {j}: {len(keypoints2)} ORB keypoints")

    # 2. Feature Matching (Brute-Force)
    raw_matches = match_features_orb_bf(descriptors1, descriptors2, ratio_thresh=0.75)
    print(f"  - Initial matches (after ratio test): {len(raw_matches)}")

    # Visualize raw matches (before RANSAC)
    if raw_matches:
        visualize_matches(img1, keypoints1, img2, keypoints2, raw_matches,
                          title=f"Raw ORB-BF Matches (Frame {i} to {j}) - {len(raw_matches)} matches")
    else:
        print(f"  - No initial matches found for Frame {i} to {j}.")
        continue # Skip RANSAC if no matches

    # 3. Outlier Rejection with RANSAC (using Fundamental Matrix)
    inlier_matches, inlier_mask, F_matrix = reject_outliers_ransac(keypoints1, keypoints2, raw_matches,
                                                                   reproj_thresh=3.0) # Adjust threshold as needed

    print(f"  - Inlier matches after RANSAC: {len(inlier_matches)}")
    if F_matrix is not None:
        print(f"  - Fundamental Matrix (F) calculated:\n{np.array2string(F_matrix, precision=4, suppress_small=True)}")
    else:
        print("  - Could not compute Fundamental Matrix (not enough inliers or degenerate case).")

    # 4. Visualize matches after RANSAC
    if inlier_matches:
        visualize_matches(img1, keypoints1, img2, keypoints2, raw_matches, inlier_mask=inlier_mask,
                          title=f"ORB-BF Matches After RANSAC (Frame {i} to {j}) - {len(inlier_matches)} inliers")
    else:
        print(f"  - No inlier matches after RANSAC for Frame {i} to {j}.")